In [9]:
import os
import pandas as pd
from IPython.display import display
import numpy as np

In [3]:
# read and check datasets in the raw file
external = pd.DataFrame()
path = os.getcwd().replace("notebooks","") + "data/raw/"
os.listdir(path)

['GNR.prj',
 '.gitkeep',
 'GNR.cpg',
 'suburb.json',
 'GNR.dbf',
 'GNR.shx',
 'datalink.txt',
 'feature.json',
 'GNR_column_names.txt',
 'GNR.shp',
 'newest.json',
 'functional.xlsx',
 'lowest_prices.json',
 'highest_price.json',
 'australian_postcodes.csv',
 'income.xlsx',
 'population.xlsx']

In [4]:
# read Australian_postcodes
postcode = pd.read_csv("../data/raw/australian_postcodes.csv")
postcode.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [5]:
# extract postcode, SA2_MAINCODE_2016, SA2_NAME_2016 from postcode to external
external['postcode'] = postcode['postcode']
external['SA2_Code'] = postcode['SA2_MAINCODE_2016']
external['SA2_Name'] = postcode['SA2_NAME_2016']
external

,postcode,SA2_Code,SA2_Name
0,200,801051049.0,Acton
1,200,801051049.0,Acton
2,800,701011002.0,Darwin City
3,800,701011002.0,Darwin City
4,801,701011002.0,Darwin City
...,...,...,...
18437,9013,305011105.0,Brisbane City
18438,9015,305011105.0,Brisbane City
18439,9464,302031038.0,Northgate - Virginia
18440,9726,309101268.0,Bundall


In [7]:
# read data, Table 1 in population
population = pd.read_excel("../data/raw/population.xlsx",sheet_name="Table 1",header=7)
#population.columns
population = population.drop(columns=['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code','SA4 name',
                                      'SA3 code', 'SA3 name',"Unnamed: 31","Unnamed: 34"])
population.head(5)

,SA2 code,SA2 name,no.,no..1,no..2,no..3,no..4,no..5,no..6,no..7,...,no..15,no..16,no..17,no..18,no..19,no..20,no..21,%,km2,persons/km2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101021007.0,Braidwood,2760.0,2811.0,2835.0,2844.0,2847.0,2965.0,3102.0,3181.0,...,3950.0,4039.0,4140.0,4211.0,4273.0,4330.0,872.0,25.2,3418.4,1.3
2,101021008.0,Karabar,9129.0,9199.0,9263.0,9277.0,9209.0,9212.0,9033.0,8994.0,...,8531.0,8526.0,8507.0,8488.0,8519.0,8546.0,-546.0,-6.0,7.0,1223.9
3,101021009.0,Queanbeyan,9717.0,9513.0,9522.0,9400.0,9595.0,9682.0,9793.0,10074.0,...,11230.0,11355.0,11447.0,11450.0,11437.0,11370.0,655.0,6.1,4.8,2387.7
4,101021010.0,Queanbeyan - East,3925.0,4073.0,4219.0,4218.0,4187.0,4319.0,4459.0,4595.0,...,4970.0,5013.0,5072.0,5117.0,5077.0,5093.0,142.0,2.9,13.0,391.7


In [8]:
# change the column name
population.columns = ['SA2_Code', 'SA2 name','2001_population','2002_population','2003_population',
                      '2004_population','2005_population','2006_population', '2007_population','2008_population',
                      '2009_population', '2010_population', '2011_population', '2012_population', '2013_population',
                      '2014_population', '2015_population', '2016_population', '2017_population','2018_population',
                      '2019_population', '2020_population', '2021_population',"Population_Change_no",
                      "Population_Change_percent","Population_Area_km2","Population_density_2021_persons/km2"]

# remove the null columns
population = population.dropna()
display(population.tail(5))
population.shape

,SA2_Code,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,2008_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
2450,801111141.0,Namadgi,12.0,11.0,11.0,10.0,10.0,9.0,15.0,22.0,...,39.0,45.0,50.0,56.0,61.0,67.0,26.0,63.4,1202.8,0.1
2451,901011001.0,Christmas Island,1442.0,1365.0,1337.0,1355.0,1380.0,1403.0,1569.0,1745.0,...,1903.0,1877.0,1849.0,1801.0,1752.0,1716.0,-451.0,-20.8,136.1,12.6
2452,901021002.0,Cocos (Keeling) Islands,600.0,568.0,558.0,573.0,588.0,590.0,575.0,568.0,...,546.0,569.0,571.0,599.0,605.0,602.0,41.0,7.3,13.7,43.9
2453,901031003.0,Jervis Bay,542.0,464.0,441.0,428.0,413.0,386.0,370.0,370.0,...,402.0,392.0,376.0,360.0,338.0,310.0,-79.0,-20.3,67.2,4.6
2454,901041004.0,Norfolk Island,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1757.0,1827.0,1920.0,1999.0,2096.0,2220.0,2220.0,0.0,38.7,57.4


(2454, 27)

In [10]:
# merge the external and population dataset together, fill null with NaN
external = pd.merge(external, population, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
0,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
1,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
2,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
3,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
4,801,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18438,9015,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18439,9464,302031038.0,Northgate - Virginia,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,...,6869.0,6900.0,7027.0,7172.0,7195.0,7169.0,575.0,8.7,6.1,1170.1
18440,9726,309101268.0,Bundall,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,...,4704.0,4885.0,4957.0,4968.0,4986.0,4930.0,566.0,13.0,3.9,1263.5


In [15]:
# read data, Table 1.4 in income
income = pd.read_excel("../data/raw/income.xlsx",sheet_name="Table 1.4",header=6)

# extract the colunms of year
income = income.drop(columns=['2014-15', '2015-16', '2016-17', '2017-18',
       '2018-19', '2014-15.1', '2015-16.1', '2016-17.1', '2017-18.1',
       '2018-19.1', '2014-15.2', '2015-16.2', '2016-17.2', '2017-18.2',
       '2018-19.2','Unnamed: 27'])
income.columns = ['SA2_Code', 'SA2 NAME','2014-15_median_income','2015-16_median_income','2016-17_median_income',
                  '2017-18_median_income','2018-19_median_income','2014-15_mean_income','2015-16_mean_income',
                  '2016-17_mean_income','2017-18_mean_income','2018-19_mean_income']

# replace np with NaN
income = income.replace('np', np.nan)
income

,SA2_Code,SA2 NAME,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,Australia,NaN,46854.0,47692.0,48360,49805.0,51389.0,61036.0,61975.0,62594.0,64246.0,65953.0
1,New South Wales,NaN,46879.0,48085.0,48700,50153.0,51818.0,62798.0,64493.0,65196.0,67200.0,68816.0
2,101021007,Braidwood,38093.0,39716.0,41288,42003.0,41593.0,47741.0,51074.0,51090.0,51594.0,51149.0
3,101021008,Karabar,54942.0,55870.0,57880,59295.0,61777.0,59548.0,61093.0,62535.0,63790.0,66335.0
4,101021009,Queanbeyan,54716.0,54999.0,55376,57848.0,60119.0,59310.0,60764.0,60958.0,62863.0,65874.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2295,801111140,ACT - South West,55294.0,62592.0,55384,61096.0,64227.0,62505.0,72858.0,70503.0,67445.0,73435.0
2296,801111141,Namadgi,NaN,NaN,49981,58498.0,40479.0,NaN,NaN,57309.0,56357.0,39278.0
2297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2298,Totals may not align with the sum of their com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# merge the exrternal with income data
external = pd.merge(external, income, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,SA2 name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,...,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,10384.0,10332.0,8682,9306.0,10433.0,21494.0,21438.0,17656.0,16835.0,19479.0
1,200,801051049.0,Acton,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,...,10384.0,10332.0,8682,9306.0,10433.0,21494.0,21438.0,17656.0,16835.0,19479.0
2,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
3,800,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
4,801,701011002.0,Darwin City,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,35116.0,35942.0,33956,37501.0,39592.0,69218.0,65963.0,65904.0,69649.0,71479.0
18438,9015,305011105.0,Brisbane City,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,...,35116.0,35942.0,33956,37501.0,39592.0,69218.0,65963.0,65904.0,69649.0,71479.0
18439,9464,302031038.0,Northgate - Virginia,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,...,54533.0,55445.0,55722,57498.0,59389.0,65369.0,66354.0,66705.0,68703.0,71806.0
18440,9726,309101268.0,Bundall,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,...,43959.0,46370.0,44066,44184.0,48078.0,67719.0,71221.0,68178.0,71310.0,75576.0


In [18]:
# read
residential_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 1",header=5)
residential_area

,SUA/SA2 code 2016,SUA/SA2 name 2016,no.,no..1,no..2,no..3,no..4,no..5,no..6
0,4001,Adelaide,7997,70705,93201,166031,55808,46801,4104
1,403041071,Aberfoyle Park,46,222,655,2111,720,581,42
2,401011001,Adelaide,1086,547,78,30,11,40,0
3,404031104,Adelaide Airport,0,1,0,0,0,0,0
4,401021004,Aldgate - Stirling,22,15,33,453,513,3612,33
...,...,...,...,...,...,...,...,...,...
1518,107031143,Warilla,18,190,4148,2241,214,110,9
1519,107011547,Windang - Primbee,2,30,558,379,180,140,2
1520,107041548,Wollongong - East,12,44,361,249,60,62,4
1521,107041549,Wollongong - West,19,146,1236,1414,433,328,10


In [ ]:
residential_area.columns = ['SA2_Code', 'SA2 name', ]